In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from WavletTransform import wavlet_transform


base = 512  # 몇 일의 데이터를 쓸 것인가
period = 42   # 몇 step내를 예측할 것인가
step = 15  # 몇 step씩 이동할 것인가

#pickle_list=['3bull_1h_uptodate.pickle', 'etf_1h.pickle', 'stock_1h.pckle']
pickle_list=['stock_1h.pickle', 'etf_1h.pickle' ]
X, Y = [],[]
for dict_file in pickle_list:
    print(dict_file)
    with open(dict_file,"rb") as fr:
        container = pickle.load(fr)

    
    for name in tqdm(container):
        
        Data = container[name]
        Data.columns=["High", "Low", "Close", "Volume"]
        Data.dropna(subset=['Close'], inplace=True) 
        Data["Volume"]=np.log(Data["Volume"]+1)        
        Data.drop(["High", "Low"], axis=1, inplace=True)
        Data.dropna(subset=['Close'], inplace=True)
        
              
            
#        date_list = Data.iloc[base-1:].loc[:].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        date_list = Data.iloc[base-1:].loc[:"2021073009"].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        #date_list = Data.iloc[base-1:].index[:-period]            #예측하려는 날짜-1
        
        # i는 예측하려는 날-1
        for i in date_list:
            
            
            close_std=Data.loc[:i].iloc[-base:]["Close"].std()    #수정종가의 표준편차
            Last_price = Data.loc[i]['Close'] + 0.0001
            y = Data.loc[i:].iloc[1:period + 1]["Close"]/Last_price - 1
            err=1
            
            for s in range(4, 3, -1):#4.5%는 변해야지
                fluct = y[abs(y) > 0.01*s+0.005]
                if np.shape(fluct)[0] > 0:
                    y=fluct.iloc[0]     #후에 어떤 추가적 변동이 있었든지 가장 먼저 변동한 것만 취급한다.
                    err=0
                    break
            if err==1:continue
            
            x = Data.loc[:i].iloc[-base:]           #예측일자 -base일
            if x.isnull().values.any() : continue   #null값 있으면 pass   
            x = (x-x.mean())/(x.std()+0.0001)        #정규화    
                        
            
            y = (np.sign(y)+1)//2  #올랐으면 1 내렸으면 0
            Y.append(y)
            X.append(x)    
#    np.save('X_%s_%d.npy' %(save_name, step), XX)
#    np.save('Y_%s_%d.npy' %(save_name, step), YY)
    


stock_1h.pickle


 54%|██████████████████████████████████████████▏                                   | 1402/2589 [09:43<07:06,  2.78it/s]

In [ ]:

XX = np.array(X)
YY = np.array(Y, dtype="uint8")

I = np.arange(np.shape(YY)[0])
np.random.shuffle(I)

XX = XX[I]
YY = YY[I]

shape = np.shape(XX)
print(shape)
print(np.shape(YY[YY==0])), print(np.shape(YY[YY==1]))
    
#save_name=dict_file.split('.')[0]

'''
#균등화
uI = YY==1
dI = YY==0

length = min(sum(uI), sum(dI))

XX = np.concatenate( (XX[uI][:length], XX[dI][:length]  ), axis=0)
YY = np.concatenate( (YY[uI][:length], YY[dI][:length]  ), axis=0)

I = np.arange(np.shape(YY)[0])

np.random.shuffle(I)

XX = XX[I]
YY = YY[I]
'''

In [ ]:
s=2
shape = np.shape(XX)[0]%s

XX = np.split(XX[shape:], s)
YY = np.split(YY[shape:], s)

for i in range(s):
    np.save('X_train2_%d.npy' %(i+1), XX[i])
    np.save('Y_train2_%d.npy' %(i+1), YY[i])
    

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from WavletTransform import wavlet_transform


base = 512  # 몇 일의 데이터를 쓸 것인가
period = 42   # 몇 step내를 예측할 것인가
step = 8  # 몇 step씩 이동할 것인가

#pickle_list=['3bull_1h_uptodate.pickle', 'etf_1h.pickle', 'stock_1h.pckle']
pickle_list=['etf_1h.pickle', 'dummy' ]
X, Y = [],[]
for dict_file in pickle_list:
    print(dict_file)
    with open(dict_file,"rb") as fr:
        container = pickle.load(fr)
    
    for name in tqdm(container):
        
        Data = container[name]
        Data.columns=["High", "Low", "Close", "Volume"]
        Data.dropna(subset=['Close'], inplace=True) 
        Data["Volume"]=np.log(Data["Volume"]+1)        
        Data.drop(["High", "Low"], axis=1, inplace=True)
        Data.dropna(subset=['Close'], inplace=True)
            
#        date_list = Data.iloc[base-1:].loc[:].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        date_list = Data.iloc[base-1:].loc[:"2021073009"].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        #date_list = Data.iloc[base-1:].index[:-period]            #예측하려는 날짜-1
        
        # i는 예측하려는 날-1
        for i in date_list:
            
            
            close_std=Data.loc[:i].iloc[-base:]["Close"].std()    #수정종가의 표준편차
            Last_price = Data.loc[i]['Close'] + 0.0001
            y = Data.loc[i:].iloc[1:period + 1]["Close"]/Last_price - 1
            err=1
            
            for s in range(4, 3, -1):#4%는 올라야지
                fluct = y[abs(y) > 0.01*s]
                if np.shape(fluct)[0] > 0:
                    y=fluct.iloc[0]     #후에 어떤 추가적 변동이 있었든지 가장 먼저 변동한 것만 취급한다.
                    err=0
                    break
            if err==1:continue
            
            x = Data.loc[:i].iloc[-base:]           #예측일자 -base일
            if x.isnull().values.any() : continue   #null값 있으면 pass             
            x = (x-x.mean())/(x.std()+0.0001)        #정규화    
                        
            
            y = (np.sign(y)+1)//2  #올랐으면 1 내렸으면 0
            Y.append(y)
            X.append(x)    

    XX = np.array(X)
    YY = np.array(Y, dtype="uint8")

    I = np.arange(np.shape(YY)[0])
    np.random.shuffle(I)

    XX = XX[I]
    YY = YY[I]

    shape = np.shape(XX)
    print(shape)
    print(np.shape(YY[YY==0])), print(np.shape(YY[YY==1]))
    
    save_name=dict_file.split('.')[0]
        
    np.save('X_%s2_%d.npy' %(save_name, step), XX)
    np.save('Y_%s2_%d.npy' %(save_name, step), YY)
    break
    


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from WavletTransform import wavlet_transform


base = 512  # 몇 일의 데이터를 쓸 것인가
period = 42   # 몇 step내를 예측할 것인가
step = 3  # 몇 step씩 이동할 것인가

#pickle_list=['3bull_1h_uptodate.pickle', 'etf_1h.pickle', 'stock_1h.pckle']
pickle_list=['3bull_1h.pickle', 'dummy' ]
X, Y = [],[]
for dict_file in pickle_list:
    print(dict_file)
    with open(dict_file,"rb") as fr:
        container = pickle.load(fr)

    
    for name in tqdm(container):
        
        Data = container[name]
        Data.columns=["High", "Low", "Close", "Volume"]
        Data.dropna(subset=['Close'], inplace=True) 
        Data["Volume"]=np.log(Data["Volume"]+1)        
        Data.drop(["High", "Low"], axis=1, inplace=True)
        Data.dropna(subset=['Close'], inplace=True)
        
              
            
#        date_list = Data.iloc[base-1:].loc[:].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        date_list = Data.iloc[base-1:].loc[:"2021073009"].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        #date_list = Data.iloc[base-1:].index[:-period]            #예측하려는 날짜-1
        
        # i는 예측하려는 날-1
        for i in date_list:
            
            
            close_std=Data.loc[:i].iloc[-base:]["Close"].std()    #수정종가의 표준편차
            Last_price = Data.loc[i]['Close'] + 0.0001
            y = Data.loc[i:].iloc[1:period + 1]["Close"]/Last_price - 1
            err=1
            
            for s in range(9, 8, -1):#9%는 올라야지
                fluct = y[abs(y) > 0.01*s]
                if np.shape(fluct)[0] > 0:
                    y=fluct.iloc[0]     #후에 어떤 추가적 변동이 있었든지 가장 먼저 변동한 것만 취급한다.
                    err=0
                    break
            if err==1:continue
            
            x = Data.loc[:i].iloc[-base:]           #예측일자 -base일
            if x.isnull().values.any() : continue   #null값 있으면 pass                   
            x = (x-x.mean())/(x.std()+0.0001)        #정규화    
                        
            
            y = (np.sign(y)+1)//2  #올랐으면 1 내렸으면 0
            Y.append(y)
            X.append(x)    

    XX = np.array(X)
    YY = np.array(Y, dtype="uint8")

    I = np.arange(np.shape(YY)[0])
    np.random.shuffle(I)

    XX = XX[I]
    YY = YY[I]

    shape = np.shape(XX)
    print(shape)
    print(np.shape(YY[YY==0])), print(np.shape(YY[YY==1]))
    
    save_name=dict_file.split('.')[0]
        
    np.save('X_%s2_%d.npy' %(save_name, step), XX)
    np.save('Y_%s2_%d.npy' %(save_name, step), YY)
    break
'''
    #균등화
    uI = YY==1
    dI = YY==0

    length = min(sum(uI), sum(dI))
        
    XX = np.concatenate( (XX[uI][:length], XX[dI][:length]  ), axis=0)
    YY = np.concatenate( (YY[uI][:length], YY[dI][:length]  ), axis=0)
'''
    


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from WavletTransform import wavlet_transform


base = 512  # 몇 일의 데이터를 쓸 것인가
period = 42   # 몇 step내를 예측할 것인가
step = 1  # 몇 step씩 이동할 것인가

#pickle_list=['3bull_1h_uptodate.pickle', 'etf_1h.pickle', 'stock_1h.pckle']
pickle_list=['3bull_1h.pickle', 'dummy' ]
X, Y = [],[]
for dict_file in pickle_list:
    print(dict_file)
    with open(dict_file,"rb") as fr:
        container = pickle.load(fr)

    
    for name in tqdm(container):
        
        Data = container[name]
        Data.columns=["High", "Low", "Close", "Volume"]
        Data.dropna(subset=['Close'], inplace=True) 
        Data["Volume"]=np.log(Data["Volume"]+1)        
        Data.drop(["High", "Low"], axis=1, inplace=True)
        Data.dropna(subset=['Close'], inplace=True)
        
              
            
#        date_list = Data.iloc[base-1:].loc[:].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        date_list = Data.iloc[base-1:].loc["2021073015":].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        #date_list = Data.iloc[base-1:].index[:-period]            #예측하려는 날짜-1
        
        # i는 예측하려는 날-1
        for i in date_list:
            
            
            close_std=Data.loc[:i].iloc[-base:]["Close"].std()    #수정종가의 표준편차
            Last_price = Data.loc[i]['Close'] + 0.0001
            y = Data.loc[i:].iloc[1:period + 1]["Close"]/Last_price - 1
            err=1
            
            for s in range(9, 7, -1):#8.5%는 올라야지
                fluct = y[abs(y) > 0.01*s+0.005]
                if np.shape(fluct)[0] > 0:
                    y=fluct.iloc[0]     #후에 어떤 추가적 변동이 있었든지 가장 먼저 변동한 것만 취급한다.
                    err=0
                    break
            if err==1:continue
            
            x = Data.loc[:i].iloc[-base:]           #예측일자 -base일
            if x.isnull().values.any() : continue   #null값 있으면 pass               
            x = (x-x.mean())/(x.std()+0.0001)        #정규화    
                        
            
            y = (np.sign(y)+1)//2  #올랐으면 1 내렸으면 0
            Y.append(y)
            X.append(x)    

    XX = np.array(X)
    YY = np.array(Y, dtype="uint8")

    I = np.arange(np.shape(YY)[0])
    np.random.shuffle(I)

    XX = XX[I]
    YY = YY[I]

    shape = np.shape(XX)
    print(shape)
    print(np.shape(YY[YY==0])), print(np.shape(YY[YY==1]))
    
    save_name=dict_file.split('.')[0]
    np.save('X_test.npy', XX)
    np.save('Y_test.npy', YY)
    break
'''
    #균등화
    uI = YY==1
    dI = YY==0

    length = min(sum(uI), sum(dI))
        
    XX = np.concatenate( (XX[uI][:length], XX[dI][:length]  ), axis=0)
    YY = np.concatenate( (YY[uI][:length], YY[dI][:length]  ), axis=0)
'''
    


In [13]:
for name in ["etf_1h_3", "train_1", "train_2"]:

    XX = np.load('X_%s.npy'%name)
    YY = np.load('Y_%s.npy'%name)
    
    s=2
    shape = np.shape(XX)[0]%s

    XX = np.split(XX[shape:], s)
    YY = np.split(YY[shape:], s)    
    
    for i in range(s):
        np.save('X_%s_%d.npy' %(name, i+1), XX[i])
        np.save('Y_%s_%d.npy' %(name, i+1), YY[i])